Install missing modules and imports

In [38]:
import sys
import subprocess
import pkg_resources

required = {'scipy', 'numpy', 'pandas', 'openpyxl', 'scikit-learn'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

Transform to PD

In [15]:
file = 'data.xlsx'
df = [pd.read_excel(file, sheet_name = sheet) for sheet in ['CAC', 'DAX', 'IBEX35TR', 'IBEX', 'SX5E', 'SX5T', 'SMI']]
for i in range(len(df)):
    df[i] = df[i].drop(0)

Helpers

In [47]:
def MSE(A, B):
    err = 0
    for i in range(len(A)):
        payoff = max(A[i] - K, 0)
        err += (payoff - B[i])**2
    return err

def g(x):
    return max(x-K, 0)

def Calculate_Kd_Ku(L):
    Ku = np.ones(len(L[0]))
    Kd = np.ones(len(L[0]))

    for i in range(len(L)):
        for j in range(len(L[i])):
            if L[i].iloc[j]['S1/S'] > Ku[j] :
                Ku[j] = L[i].iloc[j]['S1/S']
            if L[i].iloc[j]['S1/S'] < Kd[j] :
                Kd[j] = L[i].iloc[j]['S1/S']
                
    return (Ku, Kd)

def Ku_Kd_Train(Data_train):
    Kus = []
    Kds = []
    for i in range(len(Data_train)):
        Ku_, Kd_ = Calculate_Kd_Ku(Data_train[i])
        Kus.append(Ku_)
        Kds.append(Ku_)
    return Kus, Kds



Split data train test

In [24]:
'''
train_per = 0.6
train = [df[i].iloc[:math.floor(train_per*df[i].shape[0]) - math.floor(train_per*df[i].shape[0]) % 10] for i in range(len(df))]
test = [df[i].iloc[math.floor(train_per*df[i].shape[0]) - math.floor(train_per*df[i].shape[0]) % 10:(df[i].shape[0]//10)*10] for i in range(len(df))]
'''

Méthode I : Variante Min sur Dizaine

In [43]:
CAC_40 = df[0].iloc[:len(df[0])-len(df[0])%10]
n = 3
p = 0.6
CAC_40_split = np.array_split(CAC_40,10*n)
Data_train = []
Data_test = []
for i in range(len(CAC_40_split)):
    Data_train.append(CAC_40_split[i].iloc[:int(p*len(CAC_40_split[i]) - p*len(CAC_40_split[i])%10)])
    Data_test.append(CAC_40_split[i].iloc[int(p*len(CAC_40_split[i]) - p*len(CAC_40_split[i])%10):int(len(CAC_40_split[i]))])

Calcul de Ku, Kd

In [49]:
Kus = []
Kds = []
for i in range(len(Data_train)):
    Ku_, Kd_ = Calculate_Kd_Ku(Data_train[i])
    Kus.append(Ku_)
    Kds.append(Ku_)
    
print(Kus)
print(Kds)

KeyError: 0

In [20]:
def test(L, Ku, Kd):
    lamda = [(1 - Kd[i])/(Ku[i] - Kd[i]) for i in range(len(Ku))]

    V = []
    VT = []
    ST = []
    for i in range(len(L)):
        D = []
        for j in range(len(L[i])):
            if j == 0:
                D.append(L[i].iloc[j]['PX_LAST'])
            else:
                Teta = (g(Ku[j]*L[i].iloc[j]['PX_LAST'])*lamda[j] + g(Kd[j]*L[i].iloc[j]['PX_LAST'])*(1-lamda[j]))/((Ku[j]-Kd[j])*L[i].iloc[j]['PX_LAST'])
                print(Teta)
                D.append(D[-1] + Teta*L[i].iloc[j]['D_S'])
        V.append(D)
        VT.append(D[-1])
        ST.append(L[i].iloc[len(L[i])-1]['PX_LAST'])
        print(ST[-1])
        print(VT[-1])

    return MSE(ST, VT)

In [23]:
CAC_train = train_CAC40[:10]
CAC_test = train_CAC40[10:11]

val, Ku, Kd = train(CAC_train)

Val = test(CAC_test, Ku, Kd)

print(Val)

6.439233469270239
7.410538007696552
7.441789613483811
4.372969634575683
7.623282536441189
4.7631278133738775
5.3240559877536935
4.414081073104131
4.6366977969931735
5424.43
6159.20912907253
22418133.401100826
